In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:
import os
import cv2
import numpy as np

# Set the path to the directory with your cropped faces
cropped_data_path = 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Microsoft VS Code\\lfw_cropped_faces'

# A list to hold the final processed image arrays
images_with_three_or_more = []
person_labels = []

# Loop through each person's directory to count their images
print("Filtering dataset for people with 3 or more images...")
for person_name in os.listdir(cropped_data_path):
    person_dir = os.path.join(cropped_data_path, person_name)
    if os.path.isdir(person_dir):
        # Count the number of images for this person
        image_files = [f for f in os.listdir(person_dir) if f.endswith(('.jpg', '.png'))]
        
        # If the person has 3 or more images, add them to our list
        
        if len(image_files) >= 40 :
            print(f"Adding person to dataset: {person_name} with {len(image_files)} images.")
            for filename in image_files:
                img_path = os.path.join(person_dir, filename)
                img = cv2.imread(img_path)
                
                if img is not None:
                    # Convert to RGB and normalize
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = img / 255.0
                    images_with_three_or_more.append(img)
                    person_labels.append(person_name)

print("\nProcessing complete!")
print(f"Total number of images collected: {len(images_with_three_or_more)}")
print(f"Number of unique people in the dataset: {len(set(person_labels))}")

# The list 'images_with_three_or_more' now contains your preprocessed image arrays

Filtering dataset for people with 3 or more images...
Adding person to dataset: Ariel_Sharon with 76 images.
Adding person to dataset: Arnold_Schwarzenegger with 41 images.
Adding person to dataset: Colin_Powell with 235 images.
Adding person to dataset: Donald_Rumsfeld with 121 images.
Adding person to dataset: George_W_Bush with 529 images.
Adding person to dataset: Gerhard_Schroeder with 108 images.
Adding person to dataset: Gloria_Macapagal_Arroyo with 44 images.
Adding person to dataset: Hugo_Chavez with 71 images.
Adding person to dataset: Jacques_Chirac with 52 images.
Adding person to dataset: Jean_Chretien with 55 images.
Adding person to dataset: Jennifer_Capriati with 41 images.
Adding person to dataset: John_Ashcroft with 53 images.
Adding person to dataset: Junichiro_Koizumi with 60 images.
Adding person to dataset: Laura_Bush with 41 images.
Adding person to dataset: Lleyton_Hewitt with 40 images.
Adding person to dataset: Luiz_Inacio_Lula_da_Silva with 48 images.
Adding 

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


print("Converting lists to NumPy arrays...")

X_data = np.array(images_with_three_or_more)
y_labels_names = np.array(person_labels)
print("Conversion complete!")
print(f"Shape of image data: {X_data.shape}")
print(f"Shape of labels: {y_labels_names.shape}")


print("\nPreparing labels...")
le = LabelEncoder()
y_labels_encoded = le.fit_transform(y_labels_names)
y_labels_one_hot = to_categorical(y_labels_encoded)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_labels_one_hot, test_size=0.2, random_state=42
)


print("\nBuilding the balanced CNN model...")
num_classes = y_labels_one_hot.shape[1]


inputs = Input(shape=X_train.shape[1:])


x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)


embedding_output = Dense(128, activation='relu')(x)
x = Dropout(0.5)(embedding_output)


outputs = Dense(num_classes, activation='softmax')(x)

# ایجاد مدل
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\nStarting model training...")



history = model.fit(
    x=X_train,
    y=y_train,  # Add the target labels here
    epochs=40, 
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)
# 5. ذخیره مدل
model.save('my_lfw_filtered_model.h5')
print("\nModel saved successfully as 'my_lfw_filtered_model.h5'")
model.save_weights('my_face_id_model_final.weights.h5')
print("\nFinal model weights saved successfully as 'my_face_id_model_final_weights.h5'")

# 6. ایجاد مدل برای استخراج امبدینگ
embedding_model = Model(inputs=model.input, outputs=embedding_output)
print("\nEmbedding model ready for 128-dimensional feature extraction")

Converting lists to NumPy arrays...
Conversion complete!
Shape of image data: (1857, 128, 128, 3)
Shape of labels: (1857,)

Preparing labels...

Building the balanced CNN model...


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 19)             │         2,451 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,394,771 (28.21 MB)

 Trainable params: 7,394,771 (28.21 MB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 51s 360ms/step - accuracy: 0.1767 - loss: 3.3588 - val_accuracy: 0.2715 - val_loss: 2.8162
Epoch 2/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 334ms/step - accuracy: 0.2677 - loss: 2.6727 - val_accuracy: 0.2715 - val_loss: 2.6226
Epoch 3/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 331ms/step - accuracy: 0.2791 - loss: 2.6043 - val_accuracy: 0.2715 - val_loss: 2.4873
Epoch 4/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 338ms/step - accuracy: 0.2726 - loss: 2.6506 - val_accuracy: 0.2742 - val_loss: 2.4544
Epoch 5/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 339ms/step - accuracy: 0.3098 - loss: 2.4267 - val_accuracy: 0.3360 - val_loss: 2.2353
Epoch 6/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 319ms/step - accuracy: 0.3187 - loss: 2.2837 - val_accuracy: 0.3522 - val_loss: 2.0605
Epoch 7/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 343ms/step - accuracy: 0.3647 - loss: 2.1088 - val_accuracy: 0.3871 - val_loss: 1.9120
Epoch 8/40
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 330ms/step - accuracy: 0.41


Model saved successfully as 'my_lfw_filtered_model.h5'

Final model weights saved successfully as 'my_face_id_model_final_weights.h5'

Embedding model ready for 128-dimensional feature extraction


In [ ]:


test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"test accuracy{test_accuracy:.4f}")
print(f"loss function value {test_loss:.4f}")


all_predictions = model.predict(X_test, verbose=1)
predicted_classes = np.argmax(all_predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)


correct_predictions = np.sum(predicted_classes == true_classes)
total_samples = len(true_classes)
manual_accuracy = correct_predictions / total_samples

print(f"number of samples for testin {total_samples}")
print(f"number of correct predictions {correct_predictions}")
print(f"the prececion {manual_accuracy:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.7752 - loss: 0.9150
دقت مدل روی کل داده تست: 0.7688
Loss روی کل داده تست: 0.8932
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
تعداد نمونه های تست: 372
تعداد پیش‌بینی های صحیح: 286
دقت محاسبه شده دستی: 0.7688


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
X_data = np.array(images_with_three_or_more)
y_labels_names = np.array(person_labels)
print("Conversion complete!")
print(f"Shape of image data: {X_data.shape}")
print(f"Shape of labels: {y_labels_names.shape}")


print("\nPreparing labels...")
le = LabelEncoder()
y_labels_encoded = le.fit_transform(y_labels_names)
y_labels_one_hot = to_categorical(y_labels_encoded)
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_labels_one_hot, test_size=0.2, random_state=42
)
# Load the previously saved model
print("Loading the saved model...")
loaded_model = tf.keras.models.load_model('my_lfw_filtered_model.h5')
print("Model loaded successfully!")

embedding_layer = loaded_model.get_layer('dense_8') # Assuming 'dense_1' is your embedding layer
embedding_model = Model(inputs=loaded_model.input, outputs=embedding_layer.output)
print("\nEmbedding model ready for 128-dimensional feature extraction")

Conversion complete!
Shape of image data: (1857, 128, 128, 3)
Shape of labels: (1857,)

Preparing labels...
Loading the saved model...


Model loaded successfully!

Embedding model ready for 128-dimensional feature extraction


In [5]:
import numpy as np
from tensorflow.keras.models import Model


def cosine_distance(emb1, emb2):
    """
    Calculates the cosine distance between two embedding vectors.
    A smaller distance indicates greater similarity.
    """
    return 1 - np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

def find_indices_for_class(labels, target_class, num_samples=2):
    """
    Finds the indices of images belonging to a specific class.
    """
    indices = []
    for i in range(len(labels)):
        # np.argmax converts the one-hot encoded label back to a class index
        if np.argmax(labels[i]) == target_class:
            indices.append(i)
            if len(indices) == num_samples:
                break
    return indices

def compare_distances(embedding_model, X_test, y_test, num_classes_to_test=15):
    """
    Compares the average cosine distance of same-class and different-class embeddings.
    """
    same_class_distances = []
    diff_class_distances = []
    
    unique_classes = np.unique(np.argmax(y_test, axis=1))

    for i in range(num_classes_to_test):
        if i + 1 >= len(unique_classes):
            print("Warning: Not enough unique classes in y_test to run all tests.")
            break
            
        # Find two images from the same class
        target_class = unique_classes[i]
        same_indices = find_indices_for_class(y_test, target_class, num_samples=2)

        # Skip if there are not enough images for this class
        if len(same_indices) < 2:
            print(f"Skipping class {target_class}: Not enough samples.")
            continue
        
        # Extract embeddings and calculate distance
        emb1 = embedding_model.predict(X_test[same_indices[0]:same_indices[0]+1])[0]
        emb2 = embedding_model.predict(X_test[same_indices[1]:same_indices[1]+1])[0]
        distance_same = cosine_distance(emb1, emb2)
        same_class_distances.append(distance_same)

        # Find two images from different classes
        target_class_A = unique_classes[i]
        target_class_B = unique_classes[i+1] # Using the next class for comparison
        
        diff_index_A = find_indices_for_class(y_test, target_class_A, num_samples=1)[0]
        diff_index_B = find_indices_for_class(y_test, target_class_B, num_samples=1)[0]
        
        # Extract embeddings and calculate distance
        emb3 = embedding_model.predict(X_test[diff_index_A:diff_index_A+1])[0]
        emb4 = embedding_model.predict(X_test[diff_index_B:diff_index_B+1])[0]
        distance_diff = cosine_distance(emb3, emb4)
        diff_class_distances.append(distance_diff)

    # Print results
    print(f"Tested with {len(same_class_distances)} pairs of same-class images.")
    print(f"Tested with {len(diff_class_distances)} pairs of different-class images.")

    if same_class_distances:
        average_same = np.mean(same_class_distances)
        print(f"\nAverage distance for same-class images: {average_same:.4f}")
    
    if diff_class_distances:
        average_diff = np.mean(diff_class_distances)
        print(f"Average distance for different-class images: {average_diff:.4f}")

    if same_class_distances and diff_class_distances:
        average_difference = average_diff - average_same
        print(f"\nAverage difference in distances: {average_difference:.4f}")
compare_distances(embedding_model, X_test, y_test, num_classes_to_test=15)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━

In [ ]:

test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test, verbose=1)

print(f"accuracy on all of the test data {test_accuracy:.4f}")
print(f"loss on all of the test data {test_loss:.4f}")


all_predictions = loaded_model.predict(X_test, verbose=1)
predicted_classes = np.argmax(all_predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)


correct_predictions = np.sum(predicted_classes == true_classes)
total_samples = len(true_classes)
manual_accuracy = correct_predictions / total_samples

print(f"number of samples for testin {total_samples}")
print(f"number of correct predictions {correct_predictions}")
print(f"the prececion {manual_accuracy:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.7752 - loss: 0.9150
accuracy on all of the test data 0.7688
loss on all of the test data 0.8932
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step
number of samples for testin 372
number of correct predictions 286
the prececion 0.7688


In [1]:
import cv2
import os
from PIL import Image
import numpy as np
import sys

# Define the paths for input and output directories
INPUT_DIR = "C:\\Exercises\\my pics"
OUTPUT_DIR = "C:\\Exercises\\my pics\\cropped_resized_pics"
TARGET_SIZE = (128, 128)

def setup_directories():
    """
    Creates the input and output directories if they don't exist.
    """
    os.makedirs(INPUT_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Input directory '{INPUT_DIR}' and output directory '{OUTPUT_DIR}' are ready.")

def find_and_process_faces():
    """
    Iterates through images in the input directory, detects faces,
    crops and resizes them, and saves the results to the output directory.
    """
    # Load the Haar Cascade for face detection
    # This is a pre-trained model for frontal face detection.
    # The file is typically included with the opencv-python package.
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)

    if face_cascade.empty():
        print("Error: Could not load Haar Cascade file.")
        sys.exit()

    processed_count = 0
    skipped_count = 0

    # Process each file in the input directory
    for filename in os.listdir(INPUT_DIR):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Skipping non-image file: {filename}")
            skipped_count += 1
            continue

        image_path = os.path.join(INPUT_DIR, filename)

        try:
            # Read the image using OpenCV
            img_cv = cv2.imread(image_path)
            if img_cv is None:
                print(f"Warning: Could not read image at {image_path}. Skipping.")
                skipped_count += 1
                continue

            # Convert to grayscale for face detection
            gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            # The detectMultiScale function finds faces and returns a list of rectangles.
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30)
            )

            if len(faces) == 0:
                print(f"No faces found in {filename}. Skipping.")
                skipped_count += 1
                continue

            print(f"Found {len(faces)} face(s) in {filename}. Processing...")

            # Crop and save each detected face
            for i, (x, y, w, h) in enumerate(faces):
                # Crop the face region
                face_crop = img_cv[y:y+h, x:x+w]

                # Convert the OpenCV image (NumPy array) to a Pillow Image
                img_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))

                # Resize the cropped face to the target size
                img_resized = img_pil.resize(TARGET_SIZE, Image.LANCZOS)

                # Save the resized face to the output directory
                output_filename = f"{os.path.splitext(filename)[0]}_face_{i}.jpg"
                output_path = os.path.join(OUTPUT_DIR, output_filename)
                img_resized.save(output_path)
                print(f"  - Saved cropped and resized face to {output_path}")
            
            processed_count += 1

        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")
            skipped_count += 1

    print("\nProcessing complete.")
    print(f"Successfully processed {processed_count} images.")
    print(f"Skipped {skipped_count} files (no faces or invalid format).")

if __name__ == "__main__":
    setup_directories()
    
    # Check if the input directory is empty
    if not os.listdir(INPUT_DIR):
        print(f"\nPlease add your images to the '{INPUT_DIR}' directory and run the script again.")
    else:
        find_and_process_faces()

Input directory 'C:\Exercises\my pics' and output directory 'C:\Exercises\my pics\cropped_resized_pics' are ready.
Found 1 face(s) in 1.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\1_face_0.jpg
Found 1 face(s) in 2.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\2_face_0.jpg
Found 1 face(s) in 3.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\3_face_0.jpg
Found 1 face(s) in 4.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\4_face_0.jpg
Found 1 face(s) in 5.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\5_face_0.jpg
Found 1 face(s) in 6.jpg. Processing...
  - Saved cropped and resized face to C:\Exercises\my pics\cropped_resized_pics\6_face_0.jpg
Found 1 face(s) in 7.jpg. Processing...
  - Saved cropped and resized face to C:\Exerci

In [9]:
import os
import cv2
import numpy as np

# --- Configuration ---
# The directory where your pre-processed 128x128 images are saved
IMAGES_DIR = 'C:\\Exercises\\my pics\\cropped_resized_pics'
TARGET_SIZE = (128, 128)

# Check if the directory exists and has images
if not os.path.exists(IMAGES_DIR) or not os.listdir(IMAGES_DIR):
    print(f"Error: The directory '{IMAGES_DIR}' does not exist or is empty.")
    print("Please make sure you have run the preprocessing script first.")
    # Exit gracefully if the directory is not ready
    exit()

# List to hold the image data as NumPy arrays
image_list = []

print("Loading images from directory...")
for filename in os.listdir(IMAGES_DIR):
    # Check for image file extensions
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        filepath = os.path.join(IMAGES_DIR, filename)
        
        # Read the image in BGR format
        image = cv2.imread(filepath)

        # Check if the image was loaded successfully
        if image is not None:
            # Append the image array to the list
            image_list.append(image)
        else:
            print(f"Warning: Could not read image file '{filename}'. Skipping.")

# Convert the list of image arrays into a single NumPy array
images_array = np.array(image_list)

print("\n--- Process Complete ---")
print(f"Successfully loaded {len(image_list)} images.")
print(f"Final array shape: {images_array.shape}")
emb1=embedding_model.predict(images_array)
print(cosine_distance(emb1[5],emb1[1]))


Loading images from directory...

--- Process Complete ---
Successfully loaded 8 images.
Final array shape: (8, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
0.22625089
